In [1]:
!pip install numpy --upgrade

     |████████████████████████████████| 20.0MB 3.0MB/s eta 0:00:01
  Found existing installation: numpy 1.17.2
    Uninstalling numpy-1.17.2:
      Successfully uninstalled numpy-1.17.2


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.neighbors
import scipy
import importlib 
import networkx as nx
import itertools
from scipy import ndimage
#for working with tiff stacks
from skimage import io
#homebrew methods
import avulHPC
import avul

In [2]:
importlib.reload(avul)

<module 'avul' from '/home/dylan/GlobalAvulsion/avul.py'>

In [2]:
%run -i 'ImportTestDataset.py'

In [5]:
StabCrit = 4
ActLevels = []
seaRs = []
TestActLevels = []
TestseaRs = []
from IPython.core.debugger import set_trace
for x in range(9):
    Im = ImList[x]
    #get the five biggest connected components in the image
    labels_out, tra = ndimage.measurements.label(Im,structure=np.ones((3,3,3)))
    FlLab = np.ravel(labels_out)
    FlLab = FlLab[np.nonzero(FlLab)]
    if FlLab.size>0:
        u, indices = np.unique(FlLab, return_inverse=True)
        BigComLab = u[np.argmax(np.bincount(indices))]
        ComIm = labels_out * (labels_out == BigComLab)
        print(np.flatnonzero(ComIm).size)
        #Run stablesteps on filtered image points
        FiltIm = avulHPC.comfilt(ComIm)
        points = avulHPC.getpoints(FiltIm)
        EnPts1,CurActLevels, CurseaR = avulHPC.stbsteps(points,StabCrit)
        #set_trace()
        EnPts2,TestCurActLevels, TestCurseaR = avulHPC.newstbsteps(points,StabCrit)
        if EnPts1.size == EnPts2.size:
            print(np.allclose(EnPts1, EnPts2))
        ActLevels.append(CurActLevels)
        seaRs.append(CurseaR)
        TestActLevels.append(TestCurActLevels)
        TestseaRs.append(TestCurseaR)
        #np.savetxt('test'+ str(x) + 'BigComFiltV2NewImp' +'.pts',points,fmt='%1i',delimiter=' ')



319605
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
1
[15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34]
2
[15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34]
3
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34]
4
[16 17 18 19 20 21 23 24 25 27 28 29 30 31 33 34]
5
[16 17 18 19 21 24 25 27 28 30 31 33 34]
6
[16 17 18 19 24 25 27 28 30 33 34]
7
[17 24 25 30 33 34]
8
[17 24 25 30 33 34]
9
[24 25 30 33 34]
10
[24 25 30 33 34]
11
[24 25 30 33 34]
12
[24 25 30 33 34]
1
[15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34]
2
[15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34]
3
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34]
4
[16 17 18 19 20 21 23 24 25 27 28 29 30 31 33 34]
5
[16 17 18 19 21 23 24 25 27 28 30 31 33 34]
6
[16 17 18 19 24 25 27 28 30 33 34]
7
[16 17 24 25 30 33 34]
8
[17 24 25 30 33 34]
9
[24 25 30 33 34]
10
[24 25 30 33 34]
11
[24 25 30 33 34]
12
[24 25 30 33 34]
True
20

In [11]:
print(np.allclose(EnPtsLoc1.size, EnPtsLoc2.size))

True


In [12]:
ActLevels

[5, 2, 3, 7, 1, 1, 1, 1, 3]

In [14]:
TestActLevels

[5, 2, 3, 7, 1, 1, 1, 1, 3]

In [9]:
seaRs

[15.588457268119896,
 25.98076211353316,
 19.05255888325765,
 13.856406460551018,
 25.98076211353316,
 8.660254037844387,
 5.196152422706632,
 10.392304845413264,
 10.392304845413264]

In [10]:
TestseaRs

[15.588457268119896,
 25.98076211353316,
 19.05255888325765,
 13.856406460551018,
 25.98076211353316,
 8.660254037844387,
 5.196152422706632,
 10.392304845413264,
 10.392304845413264]

In [33]:
#now write test result
filtpts = avul.getpoints(FiltIm)
np.savetxt('test0' 'BigComFiltV2NewImp' +'.pts',filtpts,fmt='%1i',delimiter=' ')

In [5]:
from platform import python_version
print(python_version())

3.6.8


In [2]:
import numba
numba.__version__

'0.40.1'

In [4]:
np.version.version

'1.14.3'

In [31]:
from IPython.core.debugger import set_trace
def comfilt(ExtrIm):
    
    #for every level, the list of labels that have already been tied
    LabLst = [[] for i in range(35)]
    AllVolLabs = []
    for x in range(34):
        print(x)
        TStIm = ExtrIm[x]
        TStLab,tra = ndimage.measurements.label(TStIm,structure=np.ones((3,3)))
        labels = np.unique(TStLab[np.where(TStLab)])
        #now go through each label and find the largest component that connects to it 
        #in the next year. For each label follow this link all the way up the stack
        for y in range(labels.size):
            CurLevUsedLabs = np.asarray(LabLst[x])
            #if label has been processed previously don't analyze it
            if np.isin(labels[y], CurLevUsedLabs):
                continue
            VolLabs = []  
            labIm = TStLab * (TStLab == labels[y])
            labImXY = np.flatnonzero(labIm)
            
            #now tie the current label all the way up the stack
            for w in range(x+1,35):
                FuTStIm = ExtrIm[w]
                FuTStLab,tra = ndimage.measurements.label(FuTStIm,structure=np.ones((3,3)))
                FuTStXY = np.flatnonzero(FuTStLab)
                tra, tra, InterLocs = np.intersect1d(labImXY,FuTStXY,return_indices=True)
                OvLapLabs = np.unique(np.ravel(FuTStLab)[FuTStXY[InterLocs]])
                #hook each label up to the largest of these
                HookLabSize = 0
                HookLab = 0
                #go ahead and break if there is no component above
                if OvLapLabs.size == 0:
                    break
                for m in range(OvLapLabs.size):
                    OvlabIm = FuTStLab * (FuTStLab == OvLapLabs[m])
                    OvlabComSz = np.flatnonzero(OvlabIm).size
                    if OvlabComSz > HookLabSize:
                        HookLabSize = OvlabComSz
                        HookLab = OvLapLabs[m]
                #tie future component to current component
                if HookLab != 0:                    
                    if w == x+1:
                        LabLst[x].append(labels[y])
                        #add a unique level based identifier 
                        VolLabs.append(labels[y]+(100000*x))
                    LabLst[w].append(HookLab)
                    #add a unique level based identifier
                    VolLabs.append(HookLab+(100000*w))
                    #reset labIm one level up
                    labIm = FuTStLab * (FuTStLab == HookLab)
                    labImXY = np.flatnonzero(labIm)
            AllVolLabs.append(VolLabs)
    #now combine components that share labels 
    didMer = 1
    while didMer == 1:
        MerSw = 0
        for x in range(len(AllVolLabs)):
            CurVolLabs = AllVolLabs[x].copy()
            AllVolLabs[x] = []
            #now flatten the list of all components to check if merger happens
            FlAllVolLabs = np.concatenate(AllVolLabs)
            intLabs,tra,MerVolInds = np.intersect1d(CurVolLabs,FlAllVolLabs,return_indices=True)

            if intLabs.size > 0:
                MerSw = 1
                MerFiInd = MerVolInds[0]
                VolLabLens = list(map(len,AllVolLabs))
                VolLabLstEnLocs = np.cumsum(VolLabLens) - 1                         
                MerLstInd = np.where(MerFiInd <= VolLabLstEnLocs)[0][0]
                #need to append labels you've added to merged volume as well 
                AllVolLabs[MerLstInd].extend(CurVolLabs.copy())
                AllVolLabs[MerLstInd] = np.unique(AllVolLabs[MerLstInd]).tolist()
            else:
                AllVolLabs[x] = CurVolLabs
        #if no mergers happened break        
        if MerSw == 0:
            break
  
    #now build up the largest merged component
    FiltComLabs = AllVolLabs[np.argmax(list(map(len,AllVolLabs)))]
    StComIm = np.zeros(ExtrIm.shape)
    for x in range(len(FiltComLabs)):
        #set_trace()
        curLab = FiltComLabs[x]
        #find out what level the label is on
        Lev = np.floor(curLab/100000).astype(int)
        LevLab = curLab - (Lev*100000)
        #now get the labels at this level
        LevAllLabs,tra = ndimage.measurements.label(ExtrIm[Lev],structure=np.ones((3,3)))
        LevLabIm = LevAllLabs * (LevAllLabs == LevLab)
        LabLocs = np.nonzero(LevLabIm)
        StComIm[Lev][LabLocs] = 1
    return StComIm